# Automated Transient Determination

In [314]:
from __future__ import division, print_function, unicode_literals
import time
#import os
#import glob
#import subprocess
import numpy as np
import astropy.io.fits as fits
import sys
import datetime
import ephem

### Here we can add the below process to the existing Transients code

In [2]:
#This section tells the code where the files are. 
#data_path should point to the directory that holds the .resamp.fits files 
#produced by swarp (usually by Jack's Code)
data_path = ('./output/Temp/')

#master_file is the final processed image made from the above resamp files.
master_file = ['./output/coadd_ngc300_r.fits']

#master_weight is the weight file which corresponds to the above master file.
master_weight = ['./output/coadd_ngc300_r.weight.fits']

In [3]:
#These variables are lists of the resampled files and their corresponding weights.

resamps=sorted(glob.glob(data_path+'*.resamp.fits'))
weights=sorted(glob.glob(data_path+'*.resamp.weight.fits'))

In [10]:
#Creates a segmentation map from the master file. This will be used to generate flag files later.
#To tweak the sensitivity of this segmentation map (brightness of objects it identifies for flagging), change
#the DETECT_THRESH and ANALYSIS_THRESH values in segmentation.sex, located in the config_files directory.
#This file is a temporary file, but the code will not delete it by default, as it can be a useful analysis tool.

subprocess.call('sex '+master_file[0]+' -c ./config_files/segmentation.sex -WEIGHT_IMAGE '+master_weight[0], shell=True)
subprocess.call('mv ./Processed_Images/Temp/check.fits ./Processed_Images/Temp/segmentation_map_ref.fits', shell=True)
master_segmentation = ['./Processed_Images/Temp/segmentation_map_ref.fits']

In [12]:
#This loop creates all the difference images and their respective object, background, and aperture maps.

for i in range(0,2):#len(resamps)):
    reference=fits.open(master_file[0])
    resampled=fits.open(resamps[i])
    
    #measures the distance from the central WCS pixel to the right and to the bottom of the image, respectively.
    resdistx=(resampled[0].header['NAXIS1']-resampled[0].header['CRPIX1'])
    resdisty=(resampled[0].header['NAXIS2']-resampled[0].header['CRPIX2'])
    
    #Creates difference values that represent the start and end pixels of the resampled image on the reference image.
    ref_x_start=int(reference[0].header['CRPIX1']-resampled[0].header['CRPIX1'])
    ref_y_start=int(reference[0].header['CRPIX2']-resampled[0].header['CRPIX2'])
    ref_x_end=int(reference[0].header['CRPIX1']+resdistx)
    ref_y_end=int(reference[0].header['CRPIX2']+resdisty)
    
    #loading files into memory
    refimg=pyfits.open(master_file[0])
    resimg=pyfits.open(resamps[i])
    segimg=pyfits.open(master_segmentation[0])
    
    #converting to array
    D1=refimg[0].data
    D2=resimg[0].data
    D3=segimg[0].data
    
    #resizing and scaling pixel values
    resize_ref=D1[ref_y_start:ref_y_end,ref_x_start:ref_x_end]
    resamp_scaled=D2*resampled[0].header['FLXSCALE']
    resize_seg=D3[ref_y_start:ref_y_end,ref_x_start:ref_x_end]
    
    #doing the subtraction of the reference image from the resampled
    out_file=resamp_scaled-resize_ref
    
    #loading header information from original resampled image. This will be saved into the new images.
    head=pyfits.getheader(resamps[i])
    
    #loads the filename of the resamp image into a variable, then deletes .resamp.fits from the end of it.
    #This allows the code to write new files with the same naming format, but different file extension names.
    #If your files don't end in .resamp.fits, the -12 will need to be changed to match the number of characters
    #in the file extension.
    path=os.path.basename(resamps[i])
    new_path=path[:-12]
    
    #saves the final difference file, and the temporary resized segmentation map.
    pyfits.writeto(('./Processed_Images/'+new_path+'.difference.fits'), out_file, head)
    pyfits.writeto(('./Processed_Images/Temp/temp_segment.fits'), resize_seg, head)
    
    #produces a flag map from the temporary segmentation map.
    subprocess.call('ww -c ./config_files/default.ww -WEIGHT_NAMES '+weights[i]+',./Processed_Images/Temp/temp_segment.fits', shell=True)
    
    #does the main sextractor run on the subtracted file, producing the object, background, and aperture files.
    #this is controlled by default.sex in the config_files directory
    subprocess.call('sex ./Processed_Images/'+new_path+'.difference.fits -c ./config_files/default.sex', shell=True)
    
    #cleans up temporary files and renames background, object, and aperture files. 
    subprocess.call('rm ./Processed_Images/Temp/temp_segment.fits',shell=True)
    subprocess.call('mv ./Processed_Images/check.fits ./Processed_Images/'+new_path+'.background.fits',shell=True)
    subprocess.call('mv ./Processed_Images/check2.fits ./Processed_Images/'+new_path+'.object.fits',shell=True)
    subprocess.call('mv ./Processed_Images/check3.fits ./Processed_Images/'+new_path+'.apertures.fits',shell=True)
    subprocess.call('rm ./Processed_Images/Temp/weight.fits',shell=True)
    subprocess.call('rm ./Processed_Images/Temp/flag.fits',shell=True)
#Uncomment the next line if you would like the code to automatically clean up the master segmentation map.
#subprocess.call('rm ./Processed_Images/Temp/segmentation_map_ref.fits',shell=True)

0

# Begin Brendan's Transient checking script:

In [315]:
# this is where the code can be tacked onto the pipeline, after subtracting reference images.
# At this point we should have a combined image, with reference subtracted, where the only point sources remaining
# (in theory) are transients such as supernovae or asteroids.

# This code aims to classify such sources as either known asteroids or anomalies, which can then be manually checked.

### Run SExtractor once more to spot remaining (transient) sources:

Generate list of sources:

## Check against catalogues of known asteroids:

In [316]:
# Download latest MPCORB catalogue from:
# www.minorplanetcenter.org/iau/MPCORB/MPCORB.DAT

# Full list of paramters and how to read the data:
# http://www.minorplanetcenter.net/iau/info/MPOrbitFormat.html


# and do a manual check using PYEPHEM.

### Fiddling around with pyephem:

In [317]:
# The following 3 date entries should be indentical, but the preferred method (#3 below) is not recognised 
# for some reason...:

# date method 1:
print(ephem.Date(35497.7197916667))

# date method 2:
# Note: careful of  double parentheses; for some reason they are required.
efg = ephem.Date((1997, 11, 13, 5, 16, 30.0))
hhh = (2015, 5, 22.456)
print(ephem.Date(hhh))


# date method 3:
# this SHOULD return the same value as the above two.. not sure. will have to work around for now.
d = ephem.Date('1997/03/10 05:16:30.0')
print(d)

1997/3/10 05:16:30
2015/5/22 10:56:38


ValueError: dates must be initialized from a number, string, tuple, or datetime

In [318]:
# Checking ephem.separation() function at time of Mercury transit.

m = ephem.Uranus()
m.compute((2006, 11, 8, 21, 41, 0))

n = ephem.Sun()
n.compute((2006, 11, 8, 21, 41, 0))

print(m.ra, m.dec)
print (n.ra, n.dec)
sep = ephem.separation(m,n)
print(sep)
print(ephem.degrees(sep))
# Angular separation < 30 arcmin = transit :)
# she works!

22:50:34.32 -8:13:23.5
14:55:30.18 -16:43:34.6
114:31:07.0
114:31:07.0


In [319]:
if sep < ephem.degrees(0.00581776):
    print('the threshold is ' + str(ephem.degrees(0.00581776)))
    print('separation smaller than threshold.')
else:
    print(sep)

114:31:07.0


In [320]:
m = ephem.Mars()
d1 = ephem.Date((2009, 2, 2, 0, 0, 0))
m.compute(d1)
print(('%s  %s') %(m.ra, m.dec))

20:00:56.48  -21:30:08.1


### Import MPCORB database and split into a table:

In [321]:
# Load MPCORB.DAT data into an array:
# delimiter to split data into required columns
# Add 'skip_header=41' parameter when using real data to skip header info
# length of line = 202 characters
# 
# names = ['designation', 'abs mag', 'slope param', 'epoch', 'mean epoch anomaly', 'perihelion arg', 'longitude', /
# 'inclination', 'eccentricity', 'mean daily motion', 'semimajor axis', 'uncertainty', 'reference', 'num obs', /
# 'num opp', 'obs years / arc length', 'rms', 'coarse perturb', 'precise perturb', 'comp name', '4hex flag', / 
# 'readable designation', 'last obs']

MPCORB = np.genfromtxt('MPCORB/MPCORB_test', autostrip=True, dtype=str,  delimiter = [8,6,6,6,10,11,11,11,11,12,12,3,10,6,4,10,5,4,4,11,5,28,8])

designation = MPCORB[:,0]
abs_mag = MPCORB[:,1]
slope_param = MPCORB[:,2]
epoch = MPCORB[:,3]
mean_epoch_anomaly = MPCORB[:,4]
perihelion_arg = MPCORB[:,5]
longitude = MPCORB[:,6]
inclination = MPCORB[:,7]
eccentricity = MPCORB[:,8]
mean_daily_motion = MPCORB[:,9]
semimajor_axis = MPCORB[:,10]
uncertainty = MPCORB[:,11]
reference = MPCORB[:,12]
num_obs = MPCORB[:,13]
num_opp = MPCORB[:,14]
obs_years_arc_length = MPCORB[:,15]
rms = MPCORB[:,16]
coarse_perturb = MPCORB[:,17]
precise_perturb = MPCORB[:,18]
comp_name = MPCORB[:,19]
hex_flag =  MPCORB[:,20]
read_des = MPCORB[:,21]
last_obs = MPCORB[:,22]

### Define a function to extract the packed Epoch date format given in the MPCORB data:

In [322]:
# define dictionary of alpha to numbers for MPCORB packed dates translation:
# see  <http://www.minorplanetcenter.net/iau/info/PackedDates.html>  for more info.

def epoch_convert(date):
    
    "Converts the MPCORB Epoch from packed form to a regular YYYYMMDD.DDDD string to be split up and used later."
    
    packeddates = {'1':'1', '2':'2', '3':'3', '4':'4', '5':'5', '6':'6', '7':'7', '8':'8', '9':'9', \
                   '0':'0', 'A':'10', 'B':'11', 'C':'12', 'D':'13', 'E':'14', 'F':'15', 'G':'16', \
                   'H':'17', 'I':'18', 'J':'19', 'K':'20', 'L':'21', 'M':'22', 'N':'23', 'O':'24', \
                   'P':'25', 'Q':'26', 'R':'27', 'S':'28','T':'29', 'U':'30', 'V':'31' \
              }
    datestring = list(date[0:5])
    datestring2 = ""
    nums = "0123456789"
    
    # Conditionals to distinguish between packed dates, eg  avoid confusion between 1-Nov (11 1) /
    # and 11-Jan (1 11) when compiled back into a string (111). 
    # We convert all single-digit numeric month/day values to 2-digits (eg 6 --> 06):
    
    if date[3] in nums and date[4] in nums:
        datestring.insert(3,'0')
        datestring.insert(5,'0')
        for i in range(len(datestring)):
            char = packeddates['%s' % datestring[i]]
            datestring2 += str(char)
        if len(date) > 5:
            datestring2 += "." + date[5:]
        return datestring2 
    elif date[3] in nums and date[4] not in nums:
        datestring.insert(3,'0')
        for i in range(len(datestring)):
            char = packeddates['%s' % datestring[i]]
            datestring2 += str(char)
        if len(date) > 5:
            datestring2 += "." + date[5:]
        return datestring2
    elif date[3] not in nums and date[4] in nums:
        datestring.insert(4,'0')
        for i in range(len(datestring)):
            char = packeddates['%s' % datestring[i]]
            datestring2 += str(char)
        if len(date) > 5:
            datestring2 += "." + date[5:]
        return datestring2
    elif date[3] not in nums and date[4] not in nums:
        for i in range(len(datestring)):
            char = packeddates['%s' % datestring[i]]
            datestring2 += str(char)
        if len(date) > 5:
            datestring2 += "." + date[5:]
        return datestring2
    
    print(datestring2)

In [323]:
test1 = 'K165B5566'
test2 = epoch_convert(test1)
print(test2)
hhh = (int(test2[:4]), int(test2[4:6]), float(test2[6:]))
print(hhh)
print(ephem.Date(hhh))
print('%s' % (test2))

20160511.5566
(2016, 5, 11.5566)
2016/5/11 13:21:30
20160511.5566


In [324]:
a = int(test2[:4])
b = int(test2[5:6])
c = float(test2[6:])
print(a, b, c)
print(ephem.Date((2016, 5, 11.2566)))
ephem.Date((a, b, c))

2016 5 11.5566
2016/5/11 06:09:30


42500.0566

In [325]:
print(ephem.Date((int(test2[:4]), int(test2[4:6]), float(test2[6:]))))

2016/5/11 13:21:30


In [326]:
for packed in epoch:
    print('epoch: ' + epoch_convert(packed))

epoch: 20160113
epoch: 20160113
epoch: 20160113
epoch: 20160113
epoch: 20160113
epoch: 20160113
epoch: 20160113
epoch: 20100723


### Load some asteroid parameters into pyephem to test:

Required orbital params:

Om longitude of the ascending node 

inc = inclination to the ecliptic (plane of the Earth's orbit)

om = argument of perihelion

a = semi-major axis, or mean distance from Sun

e = eccentricity (0=circle, 0-1=ellipse, 1=parabola)

M = mean anomaly (0 at perihelion; increases uniformly with time)

## Testing Ephemeris calculation for a real asteroid, Vesta:

In [364]:
# initialise a new, blank orbital body and load orbital params:
params = MPCORB[0]
body = ephem.EllipticalBody()

#six required params for Keplerian orbit:
# Longitude of ascending node:
body._Om = float(params[6])
# Inclination:
body._inc = float(params[7])
# Arg of perihelion
body._om = float(params[5])
# Mean distance from Sun: ???????????? ACCURATE?? Semi Major Axis, technically.
body._a = float(params[10])
# Eccentricity:
body._e = float(params[8])
# Mean anomoly from perihelion:
body._M = float(params[4])
# Epoch for _M:
aster_epoch = epoch_convert('K161D')
#body._epoch_M = ephem.Date((int(aster_epoch[:4]), int(aster_epoch[4:6]), float(aster_epoch[6:])))
body._epoch_M = ephem.Date((2016, 01, 19))


huntsman = ephem.Observer()
huntsman.lon = 151.111128
huntsman.lat = -33.770281
huntsman.elevation = 50
huntsman.date = ephem.Date((2016, 1, 19, 0, 0, 0))
huntsman.epoch=ephem.J2000
#get fits header for date


print(body._epoch_M)
print(params)

2016/1/19 00:00:00
['00004' '3.20' '0.32' 'K161D' '129.52980' '151.13748' '103.84611'
 '7.14005' '0.0889223' '0.27156600' '2.3616695' '0' 'MPO358623' '6811' '98'
 '1821-2015' '0.60' 'M-p' '18h' 'MPCLINUX' '0000' '(4) Vesta' '20151215']


In [328]:
body.compute(huntsman)
print(body.a_ra, body.a_dec)
print(body.g_ra, body.g_dec)
print(body.ra, body.dec)

0:51:07.93 -1:35:30.5
0:51:56.80 -1:30:23.6
0:51:56.94 -1:30:21.3


In [330]:
date1 = ephem.Date((2016, 1, 19, 0, 0, 0))
body.compute(date1)
print(date1)
print(params[21])
print('Apparent Topocentric Positio:      RA: ' +str(body.ra) +', DEC: '+ str(body.dec))
print('Astrometric Geocentric Position:   RA: ' +str(body.a_ra) +', DEC: '+ str(body.a_dec))
print('Apparent Geocentric Position:      RA: ' +str(body.g_ra) +', DEC: '+ str(body.g_dec))

# Correct values for 19 Jan 2016 0:00:00  as per <ssd.jpl.nasa.gov/horizons.cgi>:
# 00 52 10.62 -01 26 01.5       
 
# See what values we get for our data!

# This was tested at most recent epoch. Test further away from this to check how fast errors will accumulate.

2016/1/19 00:00:00
(4) Vesta
Apparent Topocentric Positio:      RA: 0:51:56.80, DEC: -1:30:23.6
Astrometric Geocentric Position:   RA: 0:51:07.93, DEC: -1:35:30.5
Apparent Geocentric Position:      RA: 0:51:56.80, DEC: -1:30:23.6


### test reading fits header for pulling date and time of image and store as string.

In [363]:
light = '2014-09-21_83F011167_12_light.bdfw.fits'
datestr = fits.getheader(light)['DATE']
datestr

'2014-09-21T15:05:59'

In [333]:
print(ephem.Date((int(datestr[0:4]), int(datestr[5:7]), int(datestr[8:10]), int(datestr[11:13]), \
                  int(datestr[14:16]), int(datestr[17:19]))))

2014/9/21 15:05:59


### Can use this to calculate day fractionals:

In [370]:
d = ephem.Date((2014, 9, 21, 15, 5, 59))
print(d.triple())

(2014, 9, 21.629155092588917)


## Final code to check series of images against MPCORB:

In [366]:
# will need to isolate the RA/Dec of potential transients identified in SExtractor first. Then check for time/date on 
# processed image. 


#lights = (list of images with transient source data)

## Temporary data, will be replaced with actual FITS images ideally, or some other way to extract the required info
# (eg list with WCS, temp, date and time?)
lights = [['2016-01-20T00:00:00', '00:54:53.5', '-37:41:04']]

huntsman = ephem.Observer()
huntsman.lon = 149.067307
huntsman.lat = -31.274587
huntsman.elevation = 510    # Average alt. of Coona. Needs updating when dome/mount contructed.

a = time.time()
for image in lights:
    transcoords = ephem.FixedBody()   # create fixed object to hold RA dec coords??
    #transcoords = ephem.FixedBody(float(fits.getheader(image)['OBJECTRA']), float(fits.getheader(image)['OBJECTDEC']))
    transcoords._ra = image[1]
    transcoords._dec = image[2]
    transcoords.compute()
    print(image[1])
    print(transcoords._ra, transcoords._dec)
    #datestr = fits.getheader(image)['DATE']                      # get date/time from fits header
    datestr = image[0]
    huntsman.date = ephem.Date((int(datestr[0:4]), int(datestr[5:7]), int(datestr[8:10]), \
                                int(datestr[11:13]), int(datestr[14:16]), int(datestr[17:19])))
                                                                 # ^ get the date/time from the fits header
    identcount = 0                                               # initialize a new count of identified objects
    for MP in MPCORB:
        newbody = ephem.EllipticalBody()
        # Longitude of ascending node:
        newbody._Om = float(MP[6])
        # Inclination:
        newbody._inc = float(MP[7])
        # Arg of perihelion
        newbody._om = float(MP[5])
        # Mean distance from Sun: ???????????? ACCURATE?? Semi Major Axis, technically.
        newbody._a = float(MP[10])
        # Eccentricity:
        newbody._e = float(MP[8])
        # Mean anomoly from perihelion:
        newbody._M = float(MP[4])
        # Epoch = J2000:
        newbody._epoch = ephem.J2000
        # Epoch for _M:
        aster_epoch_str = epoch_convert(MP[3])
        aster_year = int(aster_epoch_str[:4])
        aster_month = int(aster_epoch_str[4:6])
        aster_day = float(aster_epoch_str[6:])
        aster_epoch = ephem.Date((aster_year, aster_month, aster_day))
        newbody._epoch_M = aster_epoch
                
        newbody.compute(huntsman)
        sep = ephem.separation(newbody, transcoords)
        print('Asteroid %s is at RA %s  Dec %s' % (MP[21], newbody.a_ra, newbody.a_dec))
        if sep < ephem.degrees(0.0872665):
            identcount += 1
            print('%s asteroid(s) found: ' %(identcount))
            print('%s is approximately %s degrees from potential transient source' % (MP[21], sep))
            # possibly also document them into a txt file?
        else:
            print('MP body %s is not within the threshold.. %s is %s degrees away from transient.' %(MP[21], MP[21], sep))
    if identcount == 0:
        print('No asteroid located within %s degrees of position RA: %s Dec: %s at time %s' %(ephem.degrees(0.0872665), transcoords._ra, transcoords._dec, huntsman.date))
    else:
        print()
        
b = time.time()
print('completed in', b-a, 'seconds')

00:54:53.5
0:54:53.50 -37:41:04.0
Asteroid (4) Vesta is at RA 0:52:30.75  Dec -1:21:10.8
MP body (4) Vesta is not within the threshold.. (4) Vesta is 36:20:17.9 degrees away from transient.
Asteroid (5) Astraea is at RA 10:11:03.49  Dec 10:20:02.1
MP body (5) Astraea is not within the threshold.. (5) Astraea is 134:14:17.1 degrees away from transient.
Asteroid (6) Hebe is at RA 12:32:28.97  Dec 5:54:07.3
MP body (6) Hebe is not within the threshold.. (6) Hebe is 147:48:20.0 degrees away from transient.
Asteroid (7) Iris is at RA 15:57:50.80  Dec -23:13:38.8
MP body (7) Iris is not within the threshold.. (7) Iris is 105:27:12.7 degrees away from transient.
Asteroid (8) Flora is at RA 16:17:50.28  Dec -16:49:26.2
MP body (8) Flora is not within the threshold.. (8) Flora is 107:36:22.5 degrees away from transient.
Asteroid (9) Metis is at RA 0:05:00.69  Dec -3:17:32.8
MP body (9) Metis is not within the threshold.. (9) Metis is 36:14:31.2 degrees away from transient.
Asteroid (10) Hygiea 

### Return anomalies:

In [ ]:
# Use pyephem, take WCS of anomalies, if pyephem of each oject in MPCORB is within a threshold (vectorize) at that time, then return object
# else, return anomaly

In [52]:
print(len('00001    3.34  0.12 K161D 181.38133   72.73324   80.32180   10.59166  0.0757544  0.21400734   2.7681117  0 MPO358623  6592 109 1801-2015 0.60 M-v 30h MPCLINUX   0000      (1) Ceres              20151128'))

202
